In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import CountVectorizer
sns.set()

In [2]:
data = pd.read_csv("./AI_Human.csv")
df = data.copy()
print(df.shape)
df.head()

(487235, 2)


,text,generated
0,Cars. Cars have been around since they became ...,0.0
1,Transportation is a large necessity in most co...,0.0
2,"""America's love affair with it's vehicles seem...",0.0
3,How often do you ride in a car? Do you drive a...,0.0
4,Cars are a wonderful thing. They are perhaps o...,0.0


In [3]:
df["generated"].value_counts()

generated
0.0    305797
1.0    181438
Name: count, dtype: int64

In [4]:
df["generated"].value_counts(normalize=True)

generated
0.0    0.627617
1.0    0.372383
Name: proportion, dtype: float64